In [1]:
import json
import os
import sys
sys.path.append('../utils')

import pandas as pd

from web3 import Web3, HTTPProvider

ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
web3 = Web3(HTTPProvider(f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"))

Get blocks where oracle updates happened: look into aggregator contract for events: AnswerUpdated

In [2]:
block_start = 13056724
block_end = web3.eth.block_number

In [3]:
crvusd_chainlink_aggregator_addr = "0xb4c4a493AB6356497713A78FFA6c60FB53517c63"
with open("../abi/crvusd_aggregator_chainlink.json", 'r') as f:
    abi = json.load(f)
crvusd_aggregator = web3.eth.contract(address=crvusd_chainlink_aggregator_addr, abi=abi)
oracle_update_filter = crvusd_aggregator.events.AnswerUpdated.createFilter(
    fromBlock=block_start,
    toBlock=block_end
)
event_output = oracle_update_filter.get_all_entries()
crvusd_oracle_update_blocks = [i['blockNumber'] for i in event_output]
len(crvusd_oracle_update_blocks)

5599

In [4]:
ethusd_chainlink_aggregator_addr = "0x37bC7498f4FF12C19678ee8fE19d713b87F6a9e6"
with open("../abi/ethusd_aggregator_chainlink.json", 'r') as f:
    abi = json.load(f)
ethusd_aggregator = web3.eth.contract(address=ethusd_chainlink_aggregator_addr, abi=abi)
oracle_update_filter = ethusd_aggregator.events.AnswerUpdated.createFilter(
    fromBlock=block_start,
    toBlock=block_end
)
event_output = oracle_update_filter.get_all_entries()
ethusd_oracle_update_blocks = [i['blockNumber'] for i in event_output]
len(ethusd_oracle_update_blocks)

8979

In [5]:
blocks = list(range(block_start, block_end))
len(blocks)

1007112

In [6]:
save_file_dir = "../data/chainlink_oracle_outputs.csv"
blocks_done = []
previous_data = pd.DataFrame()
if os.path.exists(save_file_dir):
    previous_data = pd.read_csv(save_file_dir)
    blocks_done = previous_data.block.values
previous_data

,block,crvusd,ethusd
0,13056724,2.070921,3066.626358
1,13056725,2.070921,3066.626358
2,13056726,2.070921,3066.626358
3,13056727,2.070921,3066.626358
4,13056728,2.070921,3066.626358
5,13056729,2.070921,3066.626358
6,13056730,2.070921,3066.626358
7,13056731,2.070921,3066.626358
8,13056732,2.070921,3066.626358
9,13056733,2.070921,3066.626358


In [9]:
queries_left = len(blocks)
oracle_outputs = {
    "block": [],
    "crvusd": [],
    "ethusd": []
}
crvusd_price = 0
ethusd_price = 0
for block in blocks:

    if queries_left % 10000 == 0:
        print(f"Fetching data for block {block}. Queries left: {queries_left}")

    if block in blocks_done:
        print(f"Block {block} is already queried.")
        queries_left -= 1
        continue

    if block in crvusd_oracle_update_blocks or crvusd_price == 0:
        crvusd_price = crvusd_aggregator.functions.latestAnswer().call(block_identifier=block) * 1e-8

    if block in ethusd_oracle_update_blocks or ethusd_price == 0:
        ethusd_price = ethusd_aggregator.functions.latestAnswer().call(block_identifier=block) * 1e-8

    # update dict:
    oracle_outputs['block'].append(block)
    oracle_outputs['crvusd'].append(crvusd_price)
    oracle_outputs['ethusd'].append(ethusd_price)

    queries_left -= 1

oracle_outputs

Block 13056724 is already queried.
Block 13056725 is already queried.
Block 13056726 is already queried.
Block 13056727 is already queried.
Block 13056728 is already queried.
Block 13056729 is already queried.
Block 13056730 is already queried.
Block 13056731 is already queried.
Block 13056732 is already queried.
Block 13056733 is already queried.
Block 13056734 is already queried.
Block 13056735 is already queried.
Block 13056736 is already queried.
Block 13056737 is already queried.
Block 13056738 is already queried.
Block 13056739 is already queried.
Block 13056740 is already queried.
Block 13056741 is already queried.
Block 13056742 is already queried.
Block 13056743 is already queried.
Fetching data for block 13063836. Queries left: 1000000
Fetching data for block 13073836. Queries left: 990000
Fetching data for block 13083836. Queries left: 980000
Fetching data for block 13093836. Queries left: 970000
Fetching data for block 13103836. Queries left: 960000
Fetching data for block 

{'block': [13056744,
  13056745,
  13056746,
  13056747,
  13056748,
  13056749,
  13056750,
  13056751,
  13056752,
  13056753,
  13056754,
  13056755,
  13056756,
  13056757,
  13056758,
  13056759,
  13056760,
  13056761,
  13056762,
  13056763,
  13056764,
  13056765,
  13056766,
  13056767,
  13056768,
  13056769,
  13056770,
  13056771,
  13056772,
  13056773,
  13056774,
  13056775,
  13056776,
  13056777,
  13056778,
  13056779,
  13056780,
  13056781,
  13056782,
  13056783,
  13056784,
  13056785,
  13056786,
  13056787,
  13056788,
  13056789,
  13056790,
  13056791,
  13056792,
  13056793,
  13056794,
  13056795,
  13056796,
  13056797,
  13056798,
  13056799,
  13056800,
  13056801,
  13056802,
  13056803,
  13056804,
  13056805,
  13056806,
  13056807,
  13056808,
  13056809,
  13056810,
  13056811,
  13056812,
  13056813,
  13056814,
  13056815,
  13056816,
  13056817,
  13056818,
  13056819,
  13056820,
  13056821,
  13056822,
  13056823,
  13056824,
  13056825,
  13056

In [10]:
df_queried_data = pd.DataFrame(oracle_outputs)
df_oracle_data = pd.concat([df_queried_data, previous_data])
df_oracle_data = df_oracle_data.sort_values(by='block', ascending=True).reset_index(drop=True)
df_oracle_data

,block,crvusd,ethusd
0,13056724,2.070921,3066.626358
1,13056725,2.070921,3066.626358
2,13056726,2.070921,3066.626358
3,13056727,2.070921,3066.626358
4,13056728,2.070921,3066.626358
...,...,...,...
1007107,14063831,2.868000,2415.346183
1007108,14063832,2.868000,2415.346183
1007109,14063833,2.868000,2415.346183
1007110,14063834,2.868000,2415.346183


In [11]:
df_oracle_data.to_csv(save_file_dir, index=False)